In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

: 

In [ ]:
# Load the dataset
file_path = "Goldman Sachs Stock Price History (02.05.2024-29.11.2024).csv"
df = pd.read_csv(file_path)

In [ ]:
# Data Cleaning
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Handle missing values (if any)
df.fillna(method='ffill', inplace=True)  # Forward fill missing values

In [ ]:

# Convert 'Vol.' to numeric (remove 'K' and convert to thousands)
df['Vol.'] = df['Vol.'].str.replace('K', '').astype(float) * 1000

# Convert 'Change %' to numeric (remove '%' and convert to percentage)
df['Change %'] = df['Change %'].str.replace('%', '').astype(float) / 100

# Display the cleaned dataset
print(df.head())

In [ ]:
# Data Visualization
# Plot 1: Line chart of stock prices over time
fig1 = px.line(df, x='Date', y='Price', title='Goldman Sachs Stock Price Over Time')
fig1.show()

In [ ]:

# Plot 2: Candlestick chart for stock prices
fig2 = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Price']
)])
fig2.update_layout(title='Goldman Sachs Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
fig2.show()

# Plot 3: Volume traded over time
fig3 = px.bar(df, x='Date', y='Vol.', title='Volume Traded Over Time')
fig3.show()

# Plot 4: Change % distribution
fig4 = px.histogram(df, x='Change %', title='Distribution of Daily Price Change %')
fig4.show()

# Interactive Dashboard using Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Goldman Sachs Stock Price Analysis Dashboard"),
    
    dcc.Graph(id='price-over-time', figure=fig1),
    
    dcc.Graph(id='candlestick-chart', figure=fig2),
    
    dcc.Graph(id='volume-chart', figure=fig3),
    
    dcc.Graph(id='change-distribution', figure=fig4),
    
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df['Date'].min(),
        end_date=df['Date'].max(),
        display_format='YYYY-MM-DD'
    )
])

# Callback to update graphs based on date range
@app.callback(
    [Output('price-over-time', 'figure'),
     Output('candlestick-chart', 'figure'),
     Output('volume-chart', 'figure'),
     Output('change-distribution', 'figure')],
    [Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_graphs(start_date, end_date):
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    updated_fig1 = px.line(filtered_df, x='Date', y='Price', title='Goldman Sachs Stock Price Over Time')
    
    updated_fig2 = go.Figure(data=[go.Candlestick(
        x=filtered_df['Date'],
        open=filtered_df['Open'],
        high=filtered_df['High'],
        low=filtered_df['Low'],
        close=filtered_df['Price']
    )])
    updated_fig2.update_layout(title='Goldman Sachs Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
    
    updated_fig3 = px.bar(filtered_df, x='Date', y='Vol.', title='Volume Traded Over Time')
    
    updated_fig4 = px.histogram(filtered_df, x='Change %', title='Distribution of Daily Price Change %')
    
    return updated_fig1, updated_fig2, updated_fig3, updated_fig4

# Run the dashboard
if __name__ == '__main__':
    app.run_server(debug=True)